## 결론

1. MT5는 Fine-tuning 없이 번역 추론이 거의 **불가능**하다 (self-supervised learning만 했기 때문)  
2. MT5 파라미터 수는 base 모델의 경우 약 5억 8천만, small 모델은 3억개 가량이다  

3. 장비가 부족하여 **MT5와 M2M의 공정한 비교가 불가능**하다  
    - 혹시 각 모델이 같은 Dataset을 대상으로 BLEU score를 계산한 적이 있는지 M2M, MT5 Paper를 찾아보았으나, 그런 경우는 없었다  
    - 그나마 비슷한 실험으로 toward datascience에 게재된 글이 있으나, 공정한 비교가 아니었다  
    - MT5의 경우 supervised learning이 안 된 모델인 반면, M2M은 pretrain된 모델을 가져와서 같은 양을 학습시켰기 때문에  
    - 당연히 M2M의 성능이 높을 수 밖에 없는 실험설계였다 (같은 데이터셋, 같은 epoch 횟수)  
    - https://towardsdatascience.com/comparing-facebooks-m2m-to-mt5-in-low-resources-translation-english-yoruba-ef56624d2b75
    - 그래서 직접 학습을 하여 비교하려 했으나, 시간이 너무 오래 걸려 진행이 불가능했다  
    - 노트북 GPU memory 문제 때문에 CPU로 학습을 진행해야 해서 너무 오래 걸림

### MT5에 사용된 mC4 데이터셋(101개 언어) 예시

|content-length|content-type|text|timestamp|url|
|-|-|-|-|-|
|43557|text/plain|RaptorDB - the Key Value Store - CodeProject 13,046,356 members (108,633 online)Last Visit: 31-Dec-99 18:00 Last Update: 23-Jul-17 11:31Refresh« Prev1234567891011 Next »|2017-07-23T21:31:05Z|https://www.codeproject.com/Articles/190504/RaptorDB? [...]|
|10425|text/plain|Buy Mens Troy Lee Designs Clothing at Wheelies, Free UK delivery Top Brands Tenn (62) Tenn (62) Mens Troy Lee Designs Clothing|2017-01-18T20:23:44Z|http://www.wheelies.co.uk/troy-lee-designs-mens-clothing|
|4139|text/plain|The Teachers' Lounge® (2 Ea) Nursery Rhymes Dvd Package Dimensions: Length 7.50" Width 5.38" Height 1.12" Item Number: RL382BN|2019-11-22T12:46:19Z|https://www.the-teachers-lounge.com/product/-2-ea-nurser [...]|

### 모델 크기 비교

In [54]:
from transformers import MT5ForConditionalGeneration, M2M100ForConditionalGeneration

mt5_base = MT5ForConditionalGeneration.from_pretrained('google/mt5-base')
mt5_small = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')
m2m = M2M100ForConditionalGeneration.from_pretrained('facebook/m2m100_418M')

#### 결과  
파라미터 개수 비교 (MT5 base, MT5 small, M2M 418m)

In [55]:
def model_size(m):
    return sum(p.numel() for p in m.parameters())

print('----- # of param -----')
print(f'MT5 base : {model_size(mt5_base):,}')
print(f'MT5 small : {model_size(mt5_small):,}')
print(f'M2M : {model_size(m2m):,}')

----- # of param -----
MT5 base : 582,401,280
MT5 small : 300,176,768
M2M : 486,006,784


### MT5 번역

In [51]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer

checkpoint = 'google/mt5-base'
model = MT5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5Tokenizer.from_pretrained(checkpoint)

#### 결과
**MT5는 self-supervised 학습만** 진행하고, supervised 학습을 진행하지 않아서 **fine-tuning 없이**는 번역이 제대로 되지 않는다  

In [45]:
text = 'translate English to Korean: There is an apple'
token = tokenizer(text, return_tensors='pt')
enc = model.generate(**token)
dec = tokenizer.batch_decode(enc, skip_special_tokens=True)
dec

['<extra_id_0> Korean']

#### 근거
실제 mC4 데이터 일부를 가지고 NER 추론 시 정상적으로 작동한다  
다만, 번역은 supervised 학습을 하지 않았기 때문에 위와 같이 ['<extra_id_0> Korean'] 으로 결과가 출력되는 것이다

- **ref**  
mT5 개발자 언급 : https://github.com/huggingface/transformers/issues/8704#issuecomment-732255178  
HuggingFace MT5 설명 NOTE 참조

In [42]:
text = 'Buy Mens Troy Lee Designs Clothing <extra_id_1> Wheelies'
token = tokenizer(text, return_tensors='pt')
enc = model.generate(**token)
dec = tokenizer.batch_decode(enc, skip_special_tokens=True)
dec

['<extra_id_0> Online | Cheapest Online Shopping']

### T5 번역

In [48]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

checkpoint = 't5-base'
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5Tokenizer.from_pretrained(checkpoint)

In [50]:
text = 'translate English to Korean: There is an apple'
token = tokenizer(text, return_tensors='pt')
enc = model.generate(**token)
dec = tokenizer.batch_decode(enc, skip_special_tokens=True)
dec

['Es gibt einen Apfel']

In [49]:
text = 'translate English to French: There is an apple'
token = tokenizer(text, return_tensors='pt')
enc = model.generate(**token)
dec = tokenizer.batch_decode(enc, skip_special_tokens=True)
dec

['Il y a une pomme']

#### 결과
T5의 경우 **한국어 번역을 따로** 학습해야 한다  
fine-tuning을 하지 않을 경우 한국어 번역 시도 시 독일어로 번역이 된다

## Training

In [1]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer
from transformers import AdamW
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint = 'google/mt5-small'
model = MT5ForConditionalGeneration.from_pretrained(checkpoint)
model.to(device)
tokenizer = T5Tokenizer.from_pretrained(checkpoint)
optimizer = AdamW(model.parameters(), lr=5e-4)

In [3]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, path, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.data = pd.read_json(path)[['src_text', 'tgt_text']]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        df = self.data.loc[index]
        inputs = self.tokenizer(df['src_text'], return_tensors='pt')
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(df['tgt_text'], return_tensors='pt').input_ids
        inputs['input_ids'] = inputs['input_ids'].to(device)
        inputs['attention_mask'] = inputs['attention_mask'].to(device)
        inputs['labels'] = labels.to(device)
        length = (inputs['input_ids'].shape[1], inputs['attention_mask'].shape[1], inputs['labels'].shape[1])
        return (inputs, length)

def collate(batch):
    batch, length = zip(*batch)
    ids, mask, label = zip(*length)
    max_ids, max_mask, max_label = max(ids), max(mask), max(label)
    ids, mask, label = list(ids), list(mask), list(label)
    
    ids_res, mask_res, label_res = [], [], []
    for i, sample in enumerate(batch):
        len_ids = max_ids - ids[i]
        len_mask = max_mask - mask[i]
        len_label = max_label - label[i]
        ids_tensor = torch.cat([sample['input_ids'], torch.tensor([[tokenizer.pad_token_id] * len_ids], device=device)], dim=1)
        mask_tensor = torch.cat([sample['attention_mask'], torch.tensor([[0] * len_mask], device=device)], dim=1)
        label_tensor = torch.cat([sample['labels'], torch.tensor([[tokenizer.pad_token_id] * len_label], device=device)], dim=1)
        ids_res.append(ids_tensor)
        mask_res.append(mask_tensor)
        label_res.append(label_tensor)
    ids_batch = torch.cat(ids_res, dim=0)
    mask_batch = torch.cat(mask_res, dim=0)
    label_batch = torch.cat(label_res, dim=0)
    return {'input_ids':ids_batch, 'attention_mask':mask_batch, 'labels':label_batch}
    
dataset = CustomDataset('test3.json', tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate)

In [4]:
from tqdm.notebook import tqdm

torch.set_printoptions(precision=8, sci_mode=False)
num_epochs = 10
num_training_steps = num_epochs * len(dataloader)
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    train_loss = 0
    leng = 0
    for batch in dataloader:
        batch = {k:v.to(device).long() for k, v in batch.items()}
        output = model(**batch, decoder_input_ids=batch['labels'])
        loss = output.loss
        loss.backward()
        optimizer.zero_grad()
        optimizer.step()
        train_loss += loss.item()
        leng += 1
        moving_loss = train_loss / leng
        tqdm.set_postfix(Loss=f'{moving_loss}')
        tqdm.update(1)

  0%|          | 0/50000 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 490.00 MiB (GPU 0; 6.00 GiB total capacity; 4.04 GiB already allocated; 128.50 MiB free; 4.49 GiB reserved in total by PyTorch)